In [ ]:
import glob
import numpy as np
import pygame
import os
import midi
import pretty_midi

In [ ]:
def  findRange(pattern):
    mini = 200
    maxi = -1
    for i in range(len(pattern)):
        if isinstance(pattern[i], midi.NoteEvent):
            if(pattern[i].pitch < mini):
                mini = pattern[i].pitch
            if(pattern[i].pitch > maxi):
                maxi = pattern[i].pitch
    return maxi-mini,mini

In [ ]:
def midiToNoteStateMatrix(pattern,offset):
    lowerBound = 0
    upperBound = 128
    span = upperBound-lowerBound

    timeleft = [track[0].tick for track in pattern]

    posns = [0 for track in pattern]

    statematrix = []
    time = 0

    state = [0 for x in range(span)]
    statematrix.append(state)
    condition = True
    while condition:
        if time % (pattern.resolution / 4) == (pattern.resolution / 8):
            # Crossed a note boundary. Create a new state, defaulting to holding notes
            oldstate = state
            state = [oldstate[x] for x in range(span)]
            statematrix.append(state)
        for i in range(len(timeleft)): #For each track
            if not condition:
                break
            while timeleft[i] == 0:
                track = pattern[i]
                pos = posns[i]

                evt = track[pos]
                if isinstance(evt, midi.NoteEvent):
                    if (evt.pitch < lowerBound) or (evt.pitch >= upperBound):
                        pass
                        # print "Note {} at time {} out of bounds (ignoring)".format(evt.pitch, time)
                    else:
                        if isinstance(evt, midi.NoteOffEvent) or evt.velocity == 0:
                            state[evt.pitch + offset] = 0
                        else:
                            state[evt.pitch + offset] = 1
                elif isinstance(evt, midi.TimeSignatureEvent):
                    if evt.numerator not in (2, 4):
                        # We don't want to worry about non-4 time signatures. Bail early!
                        # print "Found time signature event {}. Bailing!".format(evt)
                        out =  statematrix
                        condition = False
                        break
                try:
                    timeleft[i] = track[pos + 1].tick
                    posns[i] += 1
                except IndexError:
                    timeleft[i] = None

            if timeleft[i] is not None:
                timeleft[i] -= 1

        if all(t is None for t in timeleft):
            break

        time += 1
    statematrix = np.asarray(statematrix).tolist()
    return statematrix

In [ ]:
def noteStateMatrixToMidi(statematrix, name="example", res = 220):
    lowerBound = 0
    upperBound = 128
    span = upperBound-lowerBound

    statematrix = np.array(statematrix)
    pattern = midi.Pattern(resolution = res)
    track = midi.Track()
    pattern.append(track)
    tickscale = 55
    
    lastcmdtime = 0
    prevstate = [0 for x in range(span)]
    for time, state in enumerate(np.vstack((statematrix ,prevstate))):  
        offNotes = []
        onNotes = []
        for i in range(span):
            n = state[i]
            p = prevstate[i]
            if p == 1:
                if n == 0:
                    offNotes.append(i)
                elif n == 1:
                    pass
            elif n == 1:
                onNotes.append(i)
        for note in offNotes:
            track.append(midi.NoteOffEvent(tick=(time-lastcmdtime)*tickscale, pitch=note+lowerBound))
            lastcmdtime = time
        for note in onNotes:
            track.append(midi.NoteOnEvent(tick=(time-lastcmdtime)*tickscale, velocity=120, pitch=note+lowerBound))
            lastcmdtime = time
            
        prevstate = state
    
    eot = midi.EndOfTrackEvent(tick=1)
    track.append(eot)
    return pattern
    #midi.write_midifile("{}.mid".format(name), pattern)

In [ ]:
def splitIntoBar(stateMatrix):
    (rows,cols) = stateMatrix.shape
    ret = []
    ret2 = []
    flag = False
    for j in range(12):
        n = np.zeros((rows,cols))
        for k in range(rows):
            pos = np.argmax(stateMatrix[k])
            if pos <= 59:
                base = 48
            else:
                base = 60
            n[k , (pos+1)%12 + base] = 1
        stateMatrix = n
        i = 1
        while rows >= i*16:
            midi.write_midifile("chroma_temp.mid", noteStateMatrixToMidi(stateMatrix[(i)*16-16 : (i)*16,:]))
            midi_data = pretty_midi.PrettyMIDI('chroma_temp.mid')
            try:
                chroma = midi_data.get_chroma(fs=1.0/midi_data.get_end_time())
                ret.append([stateMatrix[(i)*16-16 : (i)*16,:]])
                ret2.append(chroma)
            except ZeroDivisionError:
                flag = True
            i += 1
        return np.asarray(ret), np.asarray(ret2), flag

In [ ]:
cnt = 0
errors = 0
ignored = 0
Cur_X = []
chromaVec = []
for root, subFolders, files in os.walk('../TheoryTab'):
        for f in files:
            if '.mid' in f:
                try:
                    pattern = midi.read_midifile(os.path.join(root, f))
                    cnt += 1
                    print "Processing file ",f,"#", cnt
                    p = midi.Pattern(resolution = pattern.resolution)
                    if len(pattern) < 2:
                        print f, 'missing melody track'
                        errors += 1
                        continue
                    p.append(pattern[0])
                    p.append(pattern[1])
                    span, mini = findRange(pattern[1])
                    if span <= 24:
                        stateMatrix = np.array(midiToNoteStateMatrix(p, 48 - mini))
                        res, res2, flag = splitIntoBar(stateMatrix)
                        Cur_X.extend(res)
                        chromaVec.extend(res2)
                        if flag == True:
                            ignored += 1
                except TypeError:
                    print f, 'raised error'
                    errors += 1

print errors, ignored, cnt

In [ ]:
Cur_X = np.asarray(Cur_X)
print Cur_X.shape
chromaVec = np.asarray(chromaVec)
print chromaVec.shape

In [ ]:
np.save('Cur_X2',Cur_X)
np.save('chromaVec2',chromaVec)

In [ ]:
pygame.init()
def play(midiFile):
    pygame.mixer.music.load(midiFile)
    pygame.mixer.music.play()

In [ ]:
#playing the resultant midi produced by the generator.
result = np.load('result.npy')
print result.shape
song = noteStateMatrixToMidi(result)
midi.write_midifile("result_tune.mid", song)
play('result_tune.mid')